# LangChain Hugging Face Agent Mini-Project

This notebook walks through building a mini-project that showcases how to combine [LangChain](https://python.langchain.com/) with a Hugging Face large language model (LLM) to create a simple AI agent. The agent can choose between multiple tools to solve tasks, which makes it an engaging classroom demonstration of *agentic* behavior.

The workflow below is Colab-friendly—you can open this notebook in Google Colab and run it top-to-bottom. Each step is intentionally documented so you can highlight the agent's reasoning process when presenting it live.

## 1. Install dependencies

Google Colab already ships with many scientific Python packages, but we still need to install LangChain and the Hugging Face Transformers stack. The `--quiet` flag keeps the output readable during demos.

In [ ]:
import os
import subprocess
import sys

packages = [
    'langchain==0.2.7',
    'langchain-community==0.2.7',
    'transformers==4.39.3',
    'accelerate==0.28.0',
    'sentencepiece==0.1.99',
]

if os.environ.get('SKIP_COLAB_INSTALL', '').lower() in {'1', 'true', 'yes'}:
    print('Skipping pip install because SKIP_COLAB_INSTALL is set.')
else:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', *packages])


## 2. Import libraries and configure the runtime

We load the required libraries and make sure LangChain prints rich traces. The helper function below performs safe mathematical evaluations so the agent can handle numeric reasoning tasks without executing arbitrary Python code.

In [ ]:

import math
import os
import re
from dataclasses import dataclass
from typing import Dict

from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool

try:
    from langchain_community.llms import HuggingFacePipeline
except ImportError:  # pragma: no cover - fallback for older LangChain releases
    try:
        from langchain.llms import HuggingFacePipeline  # type: ignore
    except ImportError as exc:  # pragma: no cover
        HuggingFacePipeline = None  # type: ignore
        print(f"Could not import HuggingFacePipeline: {exc}")

from langchain.memory import ConversationBufferMemory


USE_FAKE_LLM = bool(os.environ.get('USE_FAKE_LLM'))
os.environ.setdefault('TOKENIZERS_PARALLELISM', 'false')


def safe_eval(expression: str) -> float:
    """Safely evaluate basic arithmetic expressions for the calculator tool."""
    if not re.fullmatch(r"[0-9+\-*/().\s]+", expression):
        raise ValueError('Expression contains unsupported characters.')
    return eval(expression, {'__builtins__': {'abs': abs, 'round': round}}, {})


## 3. Load a Hugging Face LLM

We use the open-source **google/flan-t5-base** model, which runs comfortably on the free Colab T4 GPU (or even CPU). Wrapping the Transformers pipeline with LangChain's `HuggingFacePipeline` class gives us a drop-in LLM object that the agent can call.

In [ ]:
if USE_FAKE_LLM:
    print('Using offline demo mode — Hugging Face model will not be downloaded.')
    llm = None
else:
    if HuggingFacePipeline is None:
        raise ImportError('HuggingFacePipeline is required when USE_FAKE_LLM is not set.')
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

    model_name = 'google/flan-t5-base'

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    text2text_pipeline = pipeline(
        'text2text-generation',
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
    )
    llm = HuggingFacePipeline(pipeline=text2text_pipeline)


## 4. Build domain-specific tools

To demonstrate agentic behavior, we define three custom tools. Each tool solves a different class of task:

1. **Calculator** – performs arithmetic with `safe_eval`.
2. **Unit Converter** – supports a few classroom-friendly conversions.
3. **Note Summarizer** – uses the LLM directly to condense longer text snippets.

Because the tools return plain strings, they're easy to inspect while explaining how the agent reasons.

In [ ]:
CONVERSION_FACTORS: Dict[str, float] = {
    'kilometers_to_miles': 0.621371,
    'miles_to_kilometers': 1.60934,
    'celsius_to_fahrenheit': None,
}


def convert_units(query: str) -> str:
    query = query.lower().strip()
    match = re.search(r'-?\d+(?:\.\d+)?', query)
    if not match:
        raise ValueError('Please include a number in your conversion request.')
    number = float(match.group())
    if 'kilometer' in query and 'mile' in query:
        miles = number * CONVERSION_FACTORS['kilometers_to_miles']
        return f"{number} kilometers is approximately {miles:.2f} miles."
    if 'mile' in query and 'kilometer' in query:
        kilometers = number * CONVERSION_FACTORS['miles_to_kilometers']
        return f"{number} miles is approximately {kilometers:.2f} kilometers."
    if 'celsius' in query and 'fahrenheit' in query:
        fahrenheit = (number * 9 / 5) + 32
        return f"{number}°C is {fahrenheit:.1f}°F."
    raise ValueError('Unsupported conversion request. Try kilometers↔miles or Celsius↔Fahrenheit.')


def summarize_notes(text: str) -> str:
    if USE_FAKE_LLM:
        sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
        if not sentences:
            return 'Summary: No content provided.'
        summary = '; '.join(sentences[:2])
        return f'Summary: {summary}.'
    prompt = f"Summarize the following classroom note in 2 concise sentences:\n\n{text}\n\nSummary:"
    result = llm(prompt)
    return result.strip()


def evaluate_math(expression: str) -> str:
    value = safe_eval(expression)
    return f'The result is {value}.'


tools = [
    Tool(
        name='Calculator',
        func=evaluate_math,
        description="Useful for solving arithmetic expressions. Input should be a math expression like '23 * (4 + 2)'.",
    ),
    Tool(
        name='Unit Converter',
        func=convert_units,
        description='Converts simple measurements such as kilometers to miles, miles to kilometers, or Celsius to Fahrenheit.',
    ),
    Tool(
        name='Note Summarizer',
        func=summarize_notes,
        description='Summarizes provided lecture notes into a concise overview.',
    ),
]



## 5. Assemble multiple agent styles

LangChain ships with several opinionated agent templates. To highlight how design choices change the behavior, we'll spin up the classic **Zero-Shot ReAct** agent *and* the **Conversational ReAct** agent. The latter keeps a running chat history so the agent can refer back to previous turns—perfect for classroom discussions about memory.

Feel free to extend the configuration dictionary below with additional agent types if you want to experiment further in class.


In [ ]:

@dataclass
class AgentConfig:
    agent_type: AgentType
    description: str
    requires_memory: bool = False


AGENT_CHOICES = {
    'zero_shot_react': AgentConfig(
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        description='Picks tools turn-by-turn with no chat memory.',
    ),
    'conversational_react': AgentConfig(
        agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        description='Adds a conversation buffer so the agent can reference earlier turns.',
        requires_memory=True,
    ),
}


def build_agent(choice: str):
    if USE_FAKE_LLM:
        raise RuntimeError('Agents are disabled in offline demo mode.')

    config = AGENT_CHOICES[choice]
    agent_kwargs = {'handle_parsing_errors': True}

    if config.requires_memory:
        agent_kwargs['memory'] = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True,
        )

    return initialize_agent(
        tools,
        llm,
        agent=config.agent_type,
        verbose=True,
        **agent_kwargs,
    )


if USE_FAKE_LLM:
    agents = {}
    print('Skipping LangChain agent initialization in offline mode. Tool functions remain available for manual demos.')
else:
    agents = {name: build_agent(name) for name in AGENT_CHOICES}
    for name, config in AGENT_CHOICES.items():
        print(f"Loaded {name}: {config.description}")



## 6. Demonstrate three agentic tasks (and a conversational memory bonus)

Below, we run three standalone prompts through the Zero-Shot ReAct agent to showcase tool selection. Then we chat with the Conversational ReAct agent to prove it can remember earlier turns.

When presenting live, pause between prompts so learners can read the reasoning traces that LangChain prints.


In [ ]:
def run_batch(agent_executor, prompts, *, title: str):
    print(f"\n=== {title} ===")
    for prompt in prompts:
        print(f"\nUser: {prompt}")
        response = agent_executor.run(prompt)
        print(f"Agent: {response}")


zero_shot_prompts = [
    'What is (23 * 4) + 19?',
    'Convert 5 kilometers to miles.',
    'Summarize this: Photosynthesis allows plants to convert light energy into chemical energy. Chlorophyll captures sunlight and drives reactions that create glucose, releasing oxygen as a byproduct.',
]

conversational_prompts = [
    'Hi there! Please convert 3 miles to kilometers so I can note it on my class slide.',
    'Thanks for that conversion. Can you remind me of the kilometers value you just gave me?'
]

if not agents:
    print('Offline mode: skipping automated agent runs. You can still call the tool functions directly.')
else:
    run_batch(agents['zero_shot_react'], zero_shot_prompts, title='Zero-Shot ReAct tasks')
    run_batch(agents['conversational_react'], conversational_prompts, title='Conversational ReAct dialogue')



## 7. Next steps for the classroom

* Ask learners to add new tools (e.g., a date planner or trivia lookup) and observe how the agent adapts.
* Swap `google/flan-t5-base` for a more capable Hugging Face model if you have GPU resources.
* Encourage experimentation with different agent types such as the Conversational ReAct agent.

Happy teaching!